# Building SAGE-GAGE short course 2012 parallel data set
### Prepared by Gary Pavlis
The purpose of this notebook is to preserve steps needed to build the data set used for demonstration of the MsPASS framework at a short course taught at the SAGE-GAGE workshop in August 2012.  This notebook builds on some earlier work that was in a notebook called miniseed_2012.

## Load metadata 
First open a database we will call SG2012 and verify it is initially empty.


In [1]:
import os
from mspasspy.db.client import DBClient
from mspasspy.db.database import Database
hostname = os.environ.get('HOSTNAME')
dbclient = DBClient(hostname)
db=Database(dbclient,'SG2021')
cursor=db.list_collections()
for doc in cursor:
    print(doc)

{'name': 'source', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('1429cfb2-fc45-4e9b-b330-d8c785d5dcb2')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.source'}}
{'name': 'elog', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('3fd03c54-1c93-4211-8313-31fb0e7985bf')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.elog'}}
{'name': 'wf_Seismogram', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('584cc75d-f16c-4d8a-a710-8734d3dbf2af')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.wf_Seismogram'}}
{'name': 'site', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('7fa1e4aa-ed83-4d88-b137-347ec024577f')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.site'}}
{'name': 'history_object', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('981cad06-f063-40

That showed an empty database, so we can proceed.  The next step uses and import procedure borrowed from 
earlier versions of this.  I have json files I used to import the auxiliary collections (channel, site, and source) along with the nonstandard arrival collection we may or may not use in this benchmark test.  For the record these are the commands run:

```
mongoimport --host $HOSTNAME --port 27017 --db SG2021 --collection arrival --file mongodb_export/arrival.json
mongoimport --host $HOSTNAME --port 27017 --db SG2021 --collection channel --file mongodb_export/channel.json
mongoimport --host $HOSTNAME --port 27017 --db SG2021 --collection site --file mongodb_export/site.json
```
Note this intentionally omits source because I know it has an issue that needs to be fixed.  

We load arrival but not sure we will actually use it for our short course.

Now verify it worked.

In [2]:
cursor=db.channel.find({})
for doc in cursor:
    id=doc['_id']
    vang=doc['vang']
    vang +=90.0
    db.channel.update_one({'_id' : id},{'$set' : {'vang' : vang}})

In [2]:
cursor=db.list_collections()
for doc in cursor:
    print(doc)

{'name': 'site', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('7fa1e4aa-ed83-4d88-b137-347ec024577f')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.site'}}
{'name': 'arrival', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('aa51c2bc-3580-42b8-81a1-97f9949ce7ca')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.arrival'}}
{'name': 'channel', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('d3a874b2-d169-4857-8b0b-b8cf16685d97')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.channel'}}


quickly validate the counts

In [3]:
cursor=db.list_collections()
for doc in cursor:
    colname=doc['name']
    col=db[colname]
    n=col.count_documents({})
    print('Collection=',colname,' has ',n,' documents')

Collection= site  has  4518  documents
Collection= arrival  has  2995708  documents
Collection= channel  has  17780  documents


## Fixing metadata problems
The parent from which this notebook was derived (miniseed_2012 run on quakes to process the entire 2012 data set) has a long set of stuff that ended up downloading a bunch of missing station metadata in a directory called *orphanxml*.  Copied that data here to a directory with the same name using rsync.   This next box is then a variant of the parent for assimilating that data.  The original saved these as they were downloaded but here we read the files and save them.

In [4]:
from obspy import read_inventory
inv=read_inventory('orphanxml/*.xml',format='STATIONXML')
db.save_inventory(inv)

CM : CHI : 01  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   4.6295 -73.7371 3.14
loc code section coordinates:   4.6295 -73.731796 3.14
IU : COLA : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   64.873599 -147.8616 0.2
loc code section coordinates:   64.873599 -147.8616 0.08
IU : COLA : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   64.873599 -147.8616 0.2
loc code section coordinates:   64.873599 -147.8616 0.084
AK : COLD :   (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   67.226898 -150.203796 0.38
loc code section coordinates:   6

(27, 446, 259, 1902)

We didn't import the source collection from the json files because it was messed up.  We use a local copy of the quakeml file downloaded with the 2012 data in the box here.  That becomes the new source collection for this data set.

In [5]:
from obspy import read_events
cat=read_events('eventdata/events_2012.xml',format='QUAKEML')
db.save_catalog(cat)
n=db.source.count_documents({})
print('Number of source documents after save_catalog=',n)

Number of source documents after save_catalog= 961


Confirm that worked:

In [6]:
from obspy import UTCDateTime
cursor=db.source.find({}).limit(5)
for doc in cursor:
    print(doc['lat'],doc['lon'],doc['depth'],UTCDateTime(doc['time']))

-11.2308 -77.4669 77.0 2012-12-31T08:46:20.750000Z
35.2786 -36.3668 18.0 2012-12-31T01:45:01.500000Z
-12.9065 -71.0161 35.0 2012-12-30T20:49:28.550000Z
14.4746 -93.0 32.5 2012-12-30T12:21:21.770000Z
37.0194 141.2895 32.8 2012-12-29T23:05:22.840000Z


## Build wf_miniseed 
Now build the miniseed index.   Note an earlier version of this notebook has a corrupted file that created problems.  Here that file (event1.mseed) has been replaced.

In [7]:
from time import time

# This builds a file list to drive index processing
import os
import fnmatch
topdirectory='./wf'
year='2012'
dir=topdirectory+"/"+year
filelist=fnmatch.filter(os.listdir(dir),'*.mseed')

# Comment this line out if this is the first time - this is for rerun
#db.drop_collection('wf_miniseed')  
tstart=time()

for dfile in filelist:
  print(dfile)
  t0=time()
  db.index_mseed_file(dfile,dir)
  t1=time()
  dt=t1-t0
  print('Time to index file=',dfile,' = ',dt)

event6.mseed
Time to index file= event6.mseed  =  16.467503309249878
event18.mseed
Time to index file= event18.mseed  =  15.990321159362793
event13.mseed
Time to index file= event13.mseed  =  16.291953802108765
event5.mseed
Time to index file= event5.mseed  =  15.592347145080566
event11.mseed
Time to index file= event11.mseed  =  16.06053376197815
event7.mseed
Time to index file= event7.mseed  =  16.315691709518433
event12.mseed
Time to index file= event12.mseed  =  15.947712659835815
event15.mseed
Time to index file= event15.mseed  =  15.458807229995728
event16.mseed
Time to index file= event16.mseed  =  15.796416759490967
event10.mseed
Time to index file= event10.mseed  =  15.948292970657349
event19.mseed
Time to index file= event19.mseed  =  16.139742851257324
event9.mseed
Time to index file= event9.mseed  =  15.871610403060913
event17.mseed
Time to index file= event17.mseed  =  16.079205751419067
event14.mseed
Time to index file= event14.mseed  =  15.589545726776123
event3.mseed
Ti

## Check results
Quick check here for the number of entries in wf_miniseed

In [8]:
n=db.wf_miniseed.count_documents({})
print('number of documents in wf_miniseed=',n)

number of documents in wf_miniseed= 72512


## Build index and cross-ref to channel
To speed up the next step we will first build an index with net,sta,chan for the channel collection (It isn't built by default).   That will speed up the step immediately after which is to create the channel_id links between wf_miniseed and channel.

In [9]:
from pymongo import IndexModel, ASCENDING, DESCENDING
db.channel.create_index([("net",ASCENDING),("sta",ASCENDING),("chan",ASCENDING)])

'net_1_sta_1_chan_1'

Verify evid is set for some data for the algorithm below to work.

Now actually build the channel cross reference for wf_miniseed.

In [10]:
import time
from obspy import UTCDateTime
#cursor=db.wf_miniseed.find({}).limit(100)
# We print a progress line every this many documents
report_interval=100000
cursor=db.wf_miniseed.find({})
ndoc=db.wf_miniseed.count_documents({})
nprocessed=0
nset=0
t0=time.time()
for doc in cursor:
    nprocessed += 1
    net=doc['net']
    sta=doc['sta']
    chan=doc['chan']
    stime=doc['starttime']
    if "loc" in doc:
        loc = doc['loc']
        chandoc=db.get_seed_channel(net,sta,chan,loc,stime)
    else:
        chandoc=db.get_seed_channel(net,sta,chan,time=stime)    
    if not chandoc == None:
        channel_id=chandoc['_id']
        #print('Setting channel_id=',channel_id,' for ',net,':',sta,':',chan,':',loc)
        wfid=doc['_id']
        db.wf_miniseed.update_one({'_id' : wfid},{'$set' : {'channel_id' : channel_id}})
        nset += 1
    if nprocessed % report_interval == 0:
        print('Processsed ',nprocessed,' documents.  Elapsed time=',time.time()-t0)
te = time.time()
dt=te-t0
print('Total time for processing=',dt)
print('Time per update=',dt/ndoc)
print('Total documents processed=',ndoc,' Total number of ids set=',nset)

get_seed_channel:  multiple matches found for net= AZ  sta= HWB  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= AZ  sta= HWB  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= AZ  sta= HWB  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= AZ  sta= MONP2  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= AZ  sta= MONP2  and channel= BHN  with 

get_seed_channel:  multiple matches found for net= UW  sta= DAVN  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= OMAK  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= OMAK  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= OMAK  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= UMAT  and channel= BHE  with

get_seed_channel:  multiple matches found for net= CI  sta= USC  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= USC  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= USC  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= BLOW  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= BLOW  and channel= BHN  with nu

get_seed_channel:  multiple matches found for net= CI  sta= SDD  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SDD  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SDD  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SLA  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SLA  and channel= BHN  with null

get_seed_channel:  multiple matches found for net= CI  sta= MPM  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= MPM  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= MPM  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SDD  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SDD  and channel= BHN  with null

...

get_seed_channel:  multiple matches found for net= UW  sta= DAVN  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= OMAK  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= OMAK  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= OMAK  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= UMAT  and channel= BHE  with

get_seed_channel:  multiple matches found for net= UW  sta= BLOW  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= BLOW  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= BLOW  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= BRAN  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= UW  sta= BRAN  and channel= BHN  with

get_seed_channel:  multiple matches found for net= CI  sta= SMM  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SMM  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SMM  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SWS  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= SWS  and channel= BHN  with null

get_seed_channel:  multiple matches found for net= CI  sta= IKP  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= IKP  and channel= BHN  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= IKP  and channel= BHZ  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= MPM  and channel= BHE  with null loc code tested with empty string
Assuming database problem with duplicate documents in channel collection
 Returning first one found
get_seed_channel:  multiple matches found for net= CI  sta= MPM  and channel= BHN  with null

CursorNotFound: cursor id 1218242607384238754 not found, full error: {'ok': 0.0, 'errmsg': 'cursor id 1218242607384238754 not found', 'code': 43, 'codeName': 'CursorNotFound'}

## Build cross reference for site_id

In [11]:
import time
from obspy import UTCDateTime
#cursor=db.wf_miniseed.find({}).limit(100)
# We print a progress line every this many documents
report_interval=100000
cursor=db.wf_miniseed.find({})
ndoc=db.wf_miniseed.count_documents({})
nprocessed=0
nset=0
t0=time.time()
for doc in cursor:
    nprocessed += 1
    net=doc['net']
    sta=doc['sta']
    stime=doc['starttime']
    if "loc" in doc:
        loc = doc['loc']
        sitedoc=db.get_seed_site(net,sta,loc,stime)
    else:
        sitedoc=db.get_seed_site(net,sta,time=stime)    
    if not sitedoc == None:
        site_id=sitedoc['_id']
        #print('Setting channel_id=',channel_id,' for ',net,':',sta,':',chan,':',loc)
        wfid=doc['_id']
        db.wf_miniseed.update_one({'_id' : wfid},{'$set' : {'site_id' : site_id}})
        nset += 1
    if nprocessed % report_interval == 0:
        print('Processsed ',nprocessed,' documents.  Elapsed time=',time.time()-t0)
te = time.time()
dt=te-t0
print('Total time for processing=',dt)
print('Time per update=',dt/ndoc)
print('Total documents processed=',ndoc,' Total number of ids set=',nset)

get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'BZN', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'BZN', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'BZN', 'starttime': {'$lt': 1356817648.1694}, 'endtime': {'$gt': 1356817648.1694}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'CPE', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'CPE', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1

get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'WMC', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'SAO', 'loc': '00', 'starttime': {'$lt': 1356817648.169538}, 'endtime': {'$gt': 1356817648.169538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'SAO', 'loc': '00', 'starttime': {'$lt': 1356817648.169538}, 'endtime': {'$gt': 1356817648.169538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'SAO', 'loc': '00', 'starttime': {'$lt': 1356817648.169538}, 'endtime': {'$gt': 1356817648.169538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'TSCN', 'loc': '00', 'starttime': {'$lt':

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'DEC', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'DGR', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'DGR', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'DGR', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'DJJ', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 135681

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'LUG', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'LUG', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'LUG', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'MLAC', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'MLAC', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SBC', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SBC', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SBC', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SCI2', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356817648.1694999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SCI2', 'starttime': {'$lt': 1356817648.1694999}, 'endtime': {'$gt': 1356

...

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'NEE2', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356591428.5444999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'NEE2', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356591428.5444999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'NEE2', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356591428.5444999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'OSI', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356591428.5444999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'OSI', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 135

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SMM', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356591428.5444999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SMM', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356591428.5444999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SMM', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356591428.5444999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SNCC', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356591428.5444999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SNCC', 'starttime': {'$lt': 1356591428.5444999}, 'endtime': {'$gt': 1356

get_seed_site (WARNING):  query= {'net': 'IU', 'sta': 'WCI', 'loc': '00', 'starttime': {'$lt': 1356591428.569538}, 'endtime': {'$gt': 1356591428.569538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'IU', 'sta': 'WCI', 'loc': '00', 'starttime': {'$lt': 1356591428.569536}, 'endtime': {'$gt': 1356591428.569536}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'IU', 'sta': 'WCI', 'loc': '00', 'starttime': {'$lt': 1356591428.569536}, 'endtime': {'$gt': 1356591428.569536}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'IW', 'sta': 'DLMT', 'starttime': {'$lt': 1356591428.54}, 'endtime': {'$gt': 1356591428.54}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'IW', 'sta': 'DLMT', 'starttime': {'$lt': 1356591428.54}, 'endt

get_seed_site (WARNING):  query= {'net': 'NM', 'sta': 'SLM', 'starttime': {'$lt': 1356591428.56045}, 'endtime': {'$gt': 1356591428.56045}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'NM', 'sta': 'SLM', 'starttime': {'$lt': 1356591428.56045}, 'endtime': {'$gt': 1356591428.56045}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'NM', 'sta': 'SLM', 'starttime': {'$lt': 1356591428.56045}, 'endtime': {'$gt': 1356591428.56045}}
Returned  2  documents - should be exactly one
Returning first entry found


CursorNotFound: cursor id 548428486292341603 not found, full error: {'ok': 0.0, 'errmsg': 'cursor id 548428486292341603 not found', 'code': 43, 'codeName': 'CursorNotFound'}

## Build source cross reference
The following is a different algorithm than any previous approach I've used for this problem.   The approach used here depends on two assumptions about the way the data were downloaded with obspy:  (1) common events have been concatenated into event files so we can use the dfile field to group data by event, and (2) most ot the data have a start time close to the origin time of the event (it would be easy to modify this script for a constant offset from origin time, but for these data I used the origin time as a start time).    

In [12]:
import numpy as np
def print_hypo(doc):
    print(doc['lat'],doc['lon'],doc['depth'],UTCDateTime(doc['time']))
    
dtime=60.0   # plus-minus time for origin time search
# this would no scale well if the number of files where large
filellist=db.wf_miniseed.distinct('dfile')
for filename in filellist:
    query={'dfile' : filename}
    n=db.wf_miniseed.count_documents(query)
    print('File = ',filename,' has ',n,' associated documents')
    # use the median starttime for testing against source records
    cursor=db.wf_miniseed.find(query)
    idlist=list()
    stimes=[]
    for doc in cursor:
        idlist.append(doc['_id'])
        stimes.append(doc['starttime'])
    tmed=np.median(stimes)
    query={'time':{'$gt' : tmed-dtime,'$lt' : tmed+dtime}}
    n=db.source.count_documents(query)
    print('number of matching source records=',n)
    if n>0:
        srcdoc=db.source.find_one(query)
        print_hypo(srcdoc)
        # There might be a faster way to do this with update_many
        source_id=srcdoc['_id']
        for wfid in idlist:
            db.wf_miniseed.update_one({'_id' : wfid},{'$set' : {'source_id' : source_id}})

File =  event6.mseed  has  3834  associated documents
number of matching source records= 1
2.1229 -97.6847 10.0 2012-12-29T21:47:28.160000Z
File =  event18.mseed  has  3834  associated documents
number of matching source records= 1
-10.0661 -13.1969 10.0 2012-12-26T22:47:14.930000Z
File =  event13.mseed  has  3794  associated documents
number of matching source records= 1
-12.3344 -14.8715 15.0 2012-12-27T16:34:00.580000Z
File =  event5.mseed  has  3762  associated documents
number of matching source records= 1
37.0194 141.2895 32.8 2012-12-29T23:05:22.840000Z
File =  event11.mseed  has  3810  associated documents
number of matching source records= 1
-35.8925 -103.8037 10.0 2012-12-28T14:27:07.130000Z
File =  event7.mseed  has  3832  associated documents
number of matching source records= 1
1.7103 -98.0989 10.0 2012-12-29T21:37:02.000000Z
File =  event12.mseed  has  3818  associated documents
number of matching source records= 1
55.6307 164.8194 34.5 2012-12-28T13:42:34.540000Z
File = 

That algorithm worked great with a more robust solution and it ran a lot faster.  

In [1]:
import os
from mspasspy.db.database import Database
from mspasspy.db.client import DBClient

hostname = os.environ.get('HOSTNAME')
dbclient = DBClient(hostname)
db=Database(dbclient,'SG2021')

In [148]:
import pymongo
from mspasspy.algorithms.bundle import bundle_seed_data
from mspasspy.ccore.seismic import TimeSeriesEnsemble

normalize_list=['source','channel','site']
ensemble=TimeSeriesEnsemble()
cnt = cursor=db.wf_miniseed.count_documents({'dfile':{'$in':["event19.mseed"]}})
cursor=db.wf_miniseed.find({'dfile':{'$in':["event19.mseed"]}})
index = 0
for doc in cursor:
    d=db.read_data(doc,collection='wf_miniseed',normalize=normalize_list)
    if not d.is_defined('loc'):
        d.put_string('loc', "")
    print('read data for',d['sta'],d['chan'],index)
    ensemble.member.append(d)
    index += 1

read data for IUGFS BHE 0
read data for IUGFS BHN 1
read data for IUGFS BHZ 2
read data for FS01B BH1 3
read data for FS01B BH2 4
read data for FS01B BHZ 5
read data for FS05B BH1 6
read data for FS05B BH2 7
read data for FS05B BHZ 8
read data for FS06B BH1 9
read data for FS06B BH2 10
read data for FS06B BHZ 11
read data for FS09B BH1 12
read data for FS09B BH2 13
read data for FS09B BHZ 14
read data for G02B BH1 15
read data for G02B BH2 16
read data for G02B BHZ 17
read data for G03B BH1 18
read data for G03B BH2 19
read data for G03B BHZ 20
read data for G04B BH1 21
read data for G04B BH2 22
read data for G04B BHZ 23
read data for G05B BH1 24
read data for G05B BH2 25
read data for G05B BHZ 26
read data for G10B BH1 27
read data for G10B BH2 28
read data for G10B BHZ 29
read data for G11B BH1 30
read data for G11B BH2 31
read data for G11B BHZ 32
read data for G13B BH1 33
read data for G13B BH2 34
read data for G13B BHZ 35
read data for G19B BH1 36
read data for G19B BH2 37
read da

read data for BAR BHE 320
read data for BAR BHN 321
read data for BAR BHZ 322
read data for BBR BHE 323
read data for BBR BHN 324
read data for BBR BHZ 325
read data for BBS BHE 326
read data for BBS BHN 327
read data for BBS BHZ 328
read data for BC3 BHE 329
read data for BC3 BHN 330
read data for BC3 BHZ 331
read data for BCW BHE 332
read data for BCW BHN 333
read data for BCW BHZ 334
read data for BEL BHE 335
read data for BEL BHN 336
read data for BEL BHZ 337
read data for BFS BHE 338
read data for BFS BHN 339
read data for BFS BHZ 340
read data for BHP BHE 341
read data for BHP BHN 342
read data for BHP BHZ 343
read data for BLA2 BHE 344
read data for BLA2 BHN 345
read data for BLA2 BHZ 346
read data for BLY BHE 347
read data for BLY BHN 348
read data for BLY BHZ 349
read data for BOM BHE 350
read data for BOM BHN 351
read data for BOM BHZ 352
read data for BOR BHE 353
read data for BOR BHN 354
read data for BOR BHZ 355
read data for BRE BHE 356
read data for BRE BHN 357
read data

read data for PSD BHN 636
read data for PSD BHZ 637
read data for PSR BHE 638
read data for PSR BHN 639
read data for PSR BHZ 640
read data for RIN BHE 641
read data for RIN BHN 642
read data for RIN BHZ 643
read data for RIO BHE 644
read data for RIO BHN 645
read data for RIO BHZ 646
read data for RPV BHE 647
read data for RPV BHN 648
read data for RPV BHZ 649
read data for RRX BHE 650
read data for RRX BHN 651
read data for RRX BHZ 652
read data for RSS BHE 653
read data for RSS BHN 654
read data for RSS BHZ 655
read data for RUS BHE 656
read data for RUS BHN 657
read data for RUS BHZ 658
read data for RVR BHE 659
read data for RVR BHN 660
read data for RVR BHZ 661
read data for RXH BHE 662
read data for RXH BHN 663
read data for RXH BHZ 664
read data for SAL BHE 665
read data for SAL BHN 666
read data for SAL BHZ 667
read data for SBB2 BHE 668
read data for SBB2 BHN 669
read data for SBB2 BHZ 670
read data for SBC BHE 671
read data for SBC BHN 672
read data for SBC BHZ 673
read data

read data for FXWY BHN 948
read data for FXWY BHZ 949
read data for LOHW BHE 950
read data for LOHW BHN 951
read data for LOHW BHZ 952
read data for MOOW BHE 953
read data for MOOW BHN 954
read data for MOOW BHZ 955
read data for PHWY BHE 956
read data for PHWY BHN 957
read data for PHWY BHZ 958
read data for PLID BHE 959
read data for PLID BHN 960
read data for PLID BHZ 961
read data for REDW BHE 962
read data for REDW BHN 963
read data for REDW BHZ 964
read data for RWWY BHE 965
read data for RWWY BHN 966
read data for RWWY BHZ 967
read data for SMCO BHE 968
read data for SMCO BHN 969
read data for SMCO BHZ 970
read data for SNOW BHE 971
read data for SNOW BHN 972
read data for SNOW BHZ 973
read data for DAC BHE 974
read data for DAC BHN 975
read data for DAC BHZ 976
read data for ACCN BHE 977
read data for ACCN BHN 978
read data for ACCN BHZ 979
read data for ALLY BHE 980
read data for ALLY BHN 981
read data for ALLY BHZ 982
read data for BRNJ BHE 983
read data for BRNJ BHN 984
read

read data for 352A BHE 1247
read data for 352A BHN 1248
read data for 352A BHZ 1249
read data for 353A BHE 1250
read data for 353A BHN 1251
read data for 353A BHZ 1252
read data for 355A BHE 1253
read data for 355A BHN 1254
read data for 355A BHZ 1255
read data for 356A BHE 1256
read data for 356A BHN 1257
read data for 356A BHZ 1258
read data for 357A BHE 1259
read data for 357A BHN 1260
read data for 357A BHZ 1261
read data for 435B BHE 1262
read data for 435B BHN 1263
read data for 435B BHZ 1264
read data for 448A BHE 1265
read data for 448A BHN 1266
read data for 448A BHZ 1267
read data for 449A BHE 1268
read data for 449A BHN 1269
read data for 449A BHZ 1270
read data for 450A BHE 1271
read data for 450A BHN 1272
read data for 450A BHZ 1273
read data for 451A BHE 1274
read data for 451A BHN 1275
read data for 451A BHZ 1276
read data for 452A BHE 1277
read data for 452A BHN 1278
read data for 452A BHZ 1279
read data for 453A BHE 1280
read data for 453A BHN 1281
read data for 453A B

...

read data for BMO BH1 2441
read data for BMO BH2 2442
read data for BMO BHZ 2443
read data for BOZ BH1 2444
read data for BOZ BH2 2445
read data for BOZ BHZ 2446
read data for BRAL BH1 2447
read data for BRAL BH2 2448
read data for BRAL BHZ 2449
read data for BW06 BHE 2450
read data for BW06 BHN 2451
read data for BW06 BHZ 2452
read data for CBKS BH1 2453
read data for CBKS BH2 2454
read data for CBKS BHZ 2455
read data for CBN BH1 2456
read data for CBN BH2 2457
read data for CBN BHZ 2458
read data for CNNC BH1 2459
read data for CNNC BH2 2460
read data for CNNC BHZ 2461
read data for COWI BH1 2462
read data for COWI BH2 2463
read data for COWI BHZ 2464
read data for DGMT BH1 2465
read data for DGMT BH2 2466
read data for DGMT BHZ 2467
read data for DUG BH1 2468
read data for DUG BH2 2469
read data for DUG BHZ 2470
read data for ECSD BH1 2471
read data for ECSD BH2 2472
read data for ECSD BHZ 2473
read data for EGMT BH1 2474
read data for EGMT BH2 2475
read data for EGMT BHZ 2476
read

read data for A09 BHE 2742
read data for A09 BHN 2743
read data for A09 BHZ 2744
read data for A11 BHE 2745
read data for A11 BHN 2746
read data for A11 BHZ 2747
read data for A12 BHE 2748
read data for A12 BHN 2749
read data for A12 BHZ 2750
read data for A13 BHE 2751
read data for A13 BHN 2752
read data for A13 BHZ 2753
read data for A14 BHE 2754
read data for A14 BHN 2755
read data for A14 BHZ 2756
read data for A15 BHE 2757
read data for A15 BHN 2758
read data for A15 BHZ 2759
read data for A16 BHE 2760
read data for A16 BHN 2761
read data for A16 BHZ 2762
read data for A17 BH1 2763
read data for A17 BH2 2764
read data for A17 BHZ 2765
read data for A18 BHE 2766
read data for A18 BHN 2767
read data for A18 BHZ 2768
read data for A19 BHE 2769
read data for A19 BHN 2770
read data for A19 BHZ 2771
read data for A20 BHE 2772
read data for A20 BHN 2773
read data for A20 BHZ 2774
read data for A21 BHE 2775
read data for A21 BHN 2776
read data for A21 BHZ 2777
read data for A22 BHE 2778
r

read data for SS81 BHZ 3047
read data for SS82 BHE 3048
read data for SS82 BHN 3049
read data for SS82 BHZ 3050
read data for SS83 BHE 3051
read data for SS83 BHN 3052
read data for SS83 BHZ 3053
read data for LA17 BHE 3054
read data for LA17 BHN 3055
read data for LA17 BHZ 3056
read data for LA19 BHE 3057
read data for LA19 BHN 3058
read data for LA19 BHZ 3059
read data for LA21 BHE 3060
read data for LA21 BHN 3061
read data for LA21 BHZ 3062
read data for LB16 BHE 3063
read data for LB16 BHN 3064
read data for LB16 BHZ 3065
read data for LB18 BHE 3066
read data for LB18 BHN 3067
read data for LB18 BHZ 3068
read data for LB20 BHE 3069
read data for LB20 BHN 3070
read data for LB20 BHZ 3071
read data for LB22 BHE 3072
read data for LB22 BHN 3073
read data for LB22 BHZ 3074
read data for LB24 BHE 3075
read data for LB24 BHN 3076
read data for LB24 BHZ 3077
read data for LB26 BHE 3078
read data for LB26 BHN 3079
read data for LB26 BHZ 3080
read data for LC15 BHE 3081
read data for LC15 B

read data for D16 BHZ 3341
read data for D17 BHE 3342
read data for D17 BHN 3343
read data for D17 BHZ 3344
read data for D18 BHE 3345
read data for D18 BHN 3346
read data for D18 BHZ 3347
read data for D20 BHE 3348
read data for D20 BHN 3349
read data for D20 BHZ 3350
read data for D21 BHE 3351
read data for D21 BHN 3352
read data for D21 BHZ 3353
read data for D22D BHE 3354
read data for D22D BHN 3355
read data for D22D BHZ 3356
read data for D23 BHE 3357
read data for D23 BHN 3358
read data for D23 BHZ 3359
read data for D24 BHE 3360
read data for D24 BHN 3361
read data for D24 BHZ 3362
read data for D25 BHE 3363
read data for D25 BHN 3364
read data for D25 BHZ 3365
read data for D26 BHE 3366
read data for D26 BHN 3367
read data for D26 BHZ 3368
read data for D27 BHE 3369
read data for D27 BHN 3370
read data for D27 BHZ 3371
read data for D28 BHE 3372
read data for D28 BHN 3373
read data for D28 BHZ 3374
read data for D29 BHE 3375
read data for D29 BHN 3376
read data for D29 BHZ 337

read data for E19 BHE 3651
read data for E19 BHN 3652
read data for E19 BHZ 3653
read data for E20 BHE 3654
read data for E20 BHN 3655
read data for E20 BHZ 3656
read data for E21 BHE 3657
read data for E21 BHN 3658
read data for E21 BHZ 3659
read data for E22 BHE 3660
read data for E22 BHN 3661
read data for E22 BHZ 3662
read data for E23 BHE 3663
read data for E23 BHN 3664
read data for E23 BHZ 3665
read data for E24 BHE 3666
read data for E24 BHN 3667
read data for E24 BHZ 3668
read data for E25 BHE 3669
read data for E25 BHN 3670
read data for E25 BHZ 3671
read data for E26 BHE 3672
read data for E26 BHN 3673
read data for E26 BHZ 3674
read data for E27 BHE 3675
read data for E27 BHN 3676
read data for E27 BHZ 3677
read data for E28 BHE 3678
read data for E28 BHN 3679
read data for E28 BHZ 3680
read data for E29 BHE 3681
read data for E29 BHN 3682
read data for E29 BHZ 3683
read data for E30 BHE 3684
read data for E30 BHN 3685
read data for E30 BHZ 3686
read data for E31 BHE 3687
r

In [149]:
ensemble.member[0].is_defined('hang')

False

In [150]:
for i in range(len(ensemble.member)):
    if not ensemble.member[i].is_defined('hang'):
        ensemble.member[i].put_double('hang', ensemble.member[i]['channel_hang'])
    if not ensemble.member[i].is_defined('vang'):
        ensemble.member[i].put_double('vang', ensemble.member[i]['channel_vang'])

In [151]:
ensemble.member[0].is_defined('hang')

True

In [152]:
print('running bundle_seed_ensemble')
d3c=bundle_seed_data(ensemble)

running bundle_seed_ensemble


In [153]:
print(d3c.member[0].live)

True


### Window Seismogram and save

In [154]:
from obspy.geodetics import gps2dist_azimuth,kilometers2degrees
from obspy.taup import TauPyModel
from mspasspy.algorithms.window import WindowData
model = TauPyModel(model="iasp91")

for i in range(len(d3c.member)):
    stalat=d3c.member[i]['site_lat']
    stalon=d3c.member[i]['site_lon']
    srclat=d3c.member[i]['source_lat']
    srclon=d3c.member[i]['source_lon']
    depth=d3c.member[i]['source_depth']
    otime=d3c.member[i]['source_time']

    georesult=gps2dist_azimuth(srclat,srclon,stalat,stalon)
    dist=kilometers2degrees(georesult[0]/1000.0)

    arrivals = model.get_travel_times(source_depth_in_km=depth,distance_in_degree=dist,phase_list=['P'])
    if not arrivals:
        d3c.member[i].kill()
        continue
    a = arrivals[0]
    atime = a.time
    d3c.member[i].ator(otime + atime)

    d3c.member[i] = WindowData(d3c.member[i], -300, 300)
    d3c.member[i].rtoa()

In [155]:
dead_cnt = 0
for seis in d3c.member:
    if not seis.live:
        dead_cnt += 1
print('dead cnt: ', dead_cnt)

dead cnt:  501


In [156]:
dir_list = ['/tmp/data_files/' for _ in range(cnt)]
dfile_list = ['file19' for _ in range(cnt)]
db.save_ensemble_data(d3c, storage_mode='file', dir_list=dir_list, dfile_list=dfile_list, data_tag='rawdata')

In [157]:
db.elog.count_documents({})

85

In [158]:
elog_docs=db.elog.find({})
for doc in elog_docs:
    print(doc['wf_Seismogram_id'])
    db.wf_Seismogram.delete_one({'_id': doc['wf_Seismogram_id']})

61076e4bad4e0df4015f9181
61076e4bad4e0df4015f9144
61076e4bad4e0df4015f9106
61076e4bad4e0df4015f90e4
61076e4aad4e0df4015f8f75
61076e45ad4e0df4015f894a
61076e45ad4e0df4015f8949
610d546d5719f54c84c1290a
610d546d5719f54c84c1290b
610d546d5719f54c84c1290c
610d546d5719f54c84c1290d
61076e1aad4e0df4015f62dc
61076e15ad4e0df4015f5cb4
61076e15ad4e0df4015f5cb3
61076e14ad4e0df4015f5bec
61076e6aad4e0df4015fad34
61076e65ad4e0df4015fa70c
61076e65ad4e0df4015fa70b
61076e65ad4e0df4015fa644
61076e63ad4e0df4015fa3c8
61076ec9ad4e0df4016006c7
61076ec4ad4e0df4016000a2
61076ec4ad4e0df4016000a1
61076ec4ad4e0df4016000a0
61076ec2ad4e0df4015ffd69
61076ea9ad4e0df4015fe8ef
61076ea4ad4e0df4015fe2ca
61076ea4ad4e0df4015fe2c9
61076ea4ad4e0df4015fe2c8
61076ea4ad4e0df4015fe1cd
61076ea2ad4e0df4015fdf8b
61076e5aad4e0df4015f9e39
61076e55ad4e0df4015f981d
61076e55ad4e0df4015f981c
61076e55ad4e0df4015f981b
61076e55ad4e0df4015f980d
61076e54ad4e0df4015f9725
61076e52ad4e0df4015f94e6
61076e7aad4e0df4015fbc1b
61076e75ad4e0df4015fb5f9


In [159]:
db.wf_Seismogram.update_many({}, {'$unset': {'format': 1}})

In [28]:
import time
doc=db.wf_Seismogram.find_one({'dfile':'file5'})
normlist=['source','site']
t0=time.time()
d=db.read_data(doc,collection='wf_Seismogram',normalize=normlist)
t1=time.time()
print('read_data time: ', t1-t0)

36003
read_data time:  2.5879456996917725


### Hardcode vang and hang

In [26]:
cursor=db.channel.find({'chan':{'$in':['HHE','BHE','HH2','BH2']}})
for doc in cursor:
    db.channel.update_one({'_id' : doc['_id']},{'$set' : {'vang' : 90.0, 'hang': 90.0}})

In [28]:
cursor=db.channel.find({'chan':{'$in':['HHN','BHN','HH1','BH1']}})
for doc in cursor:
    db.channel.update_one({'_id' : doc['_id']},{'$set' : {'vang' : 90.0, 'hang': 0.0}})

In [24]:
cursor=db.channel.find({'chan':{'$in':['HHZ','BHZ']}})
for doc in cursor:
    db.channel.update_one({'_id' : doc['_id']},{'$set' : {'vang' : 0.0, 'hang': 0.0}})